# Example 2: CZ gate
In this example we analyze what happens if two polarized indistinguishable 
photons mix at the general beam splitter with reflectances $T_h$ and $T_v$ for
each polarization. Then we attenuate the modues again with another beam splitters.


In [27]:
import json
import numpy as np
import sympy as sp
from json_netlist import instantiate_netlist_components, sp_calculate_effective_matrix
from two_photon_conversion import sp_construct_operator, sp_create_filter_matrix

mode numbering:
* 0 : path A, polarization H
* 1 : path A, polarization V
* 2 : path B, polarization H
* 3 : path B, polarization V
it is same for input and output

In [28]:
netlist_json_string = """
{
    "BSAtten1" : {
      "type" : "BS2pol",
      "input_modes" : [0,1,null,null],
      "output_modes" : [0,1,null,null],
      "layer" : 0,
      "arguments" : ["Th1","Tv1"],
      "kw_args" : {"sympy" : true}
    },
    "BSAtten2" : {
      "type" : "BS2pol",
      "input_modes" : [2,3,null,null],
      "output_modes" : [2,3,null,null],
      "layer" : 0,
      "arguments" : ["Th2","Tv2"],
      "kw_args" : {"sympy" : true}
    },     
    "CentralBS" : {
      "type" : "BS2pol",
      "input_modes" : [0,1,2,3],
      "output_modes" : [0,1,2,3],
      "layer" : 1,
      "arguments" : ["Th","Tv"],
      "kw_args" : {"sympy" : true}
    }
}
"""
netlist = json.loads(netlist_json_string)
instances = instantiate_netlist_components(netlist)
single_photon_map = sp_calculate_effective_matrix(instances)
single_photon_map #print single photon map

Matrix([
[     sqrt(Th)*sqrt(Th1),                       0, sqrt(Th2)*sqrt(1 - Th),                      0],
[                      0,      sqrt(Tv)*sqrt(Tv1),                      0, sqrt(Tv2)*sqrt(1 - Tv)],
[-sqrt(Th1)*sqrt(1 - Th),                       0,     sqrt(Th)*sqrt(Th2),                      0],
[                      0, -sqrt(Tv1)*sqrt(1 - Tv),                      0,     sqrt(Tv)*sqrt(Tv2)]])

how computational basis state are encoded:
* |00> : signal photon in mode 0 (path A, polarization H), idler photon in mode 2 (path B, polarization H)
* |01> : signal photon in mode 0 (path A, polarization H), idler photon in mode 3 (path B, polarization V)
* |10> : signal photon in mode 1 (path A, polarization V), idler photon in mode 2 (path B, polarization H)
* |11> : signal photon in mode 1 (path A, polarization V), idler photon in mode 3 (path B, polarization V)
the output encoding is the same.

In [29]:
allowed_inputs = [
    (0,2),
    (0,3),
    (1,2),
    (1,3)
]
allowed_outputs = [
    (0,2),
    (0,3),
    (1,2),
    (1,3)
]
n_map_rows = single_photon_map.shape[0]
filter_matrix = sp_create_filter_matrix(allowed_outputs, n_map_rows, True, True)
two_photon_operator = sp_construct_operator(single_photon_map, allowed_inputs, filter_matrix)
two_photon_operator #print single photon map

Matrix([
[1.0*Th*sqrt(Th1)*sqrt(Th2) - 1.0*sqrt(Th1)*sqrt(Th2)*(1 - Th),                                                  0,                                                  0,                                                             0],
[                                                            0,          1.0*sqrt(Th)*sqrt(Th1)*sqrt(Tv)*sqrt(Tv2), -1.0*sqrt(Th2)*sqrt(Tv1)*sqrt(1 - Th)*sqrt(1 - Tv),                                                             0],
[                                                            0, -1.0*sqrt(Th1)*sqrt(Tv2)*sqrt(1 - Th)*sqrt(1 - Tv),          1.0*sqrt(Th)*sqrt(Th2)*sqrt(Tv)*sqrt(Tv1),                                                             0],
[                                                            0,                                                  0,                                                  0, 1.0*Tv*sqrt(Tv1)*sqrt(Tv2) - 1.0*sqrt(Tv1)*sqrt(Tv2)*(1 - Tv)]])

In [30]:
sp.N(two_photon_operator.subs([('Th', 1), ('Tv', 1/3), ('Th1', 1/3), ('Tv1', 1), ('Th2', 1/3), ('Tv2', 1)] ),chop=1e-12)

Matrix([
[0.333333333333333,                 0,                 0,                  0],
[                0, 0.333333333333333,                 0,                  0],
[                0,                 0, 0.333333333333333,                  0],
[                0,                 0,                 0, -0.333333333333333]])

In [31]:
#create numpy function describing the operator
foo = sp.lambdify([sp.Symbol('Th'), sp.Symbol('Tv'), sp.Symbol('Th1'), sp.Symbol('Tv1'), sp.Symbol('Th2'), sp.Symbol('Tv2')], two_photon_operator, modules = ['numpy'])
foo(1,0.3333, 0.3333, 1, 0.3333, 1)

array([[ 0.3333,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.3333, -0.    ,  0.    ],
       [ 0.    , -0.    ,  0.3333,  0.    ],
       [ 0.    ,  0.    ,  0.    , -0.3334]])